In [7]:
data_path = 'D:/workspace/03.UseDataAnalysis/04.AI 서비스 개발 프로젝트/gpt/gptdata/melon_ballad.csv'
model_save = 'D:/workspace/03.UseDataAnalysis/04.AI 서비스 개발 프로젝트/gpt/gptdata/8.발라드가사.csv'
output_path = 'gpt/result/발라드가사'
input =  '제주 애월읍 해변위에 앉아 사진을 찍는다.'

# 전처리

In [8]:
def cleasing(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail주소제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub(pattern='\n[0-9]', repl='', string=text)
    # pattern = '[^\w\s]' # 특수기호제거
    # text = re.sub(pattern=pattern, repl='', string=text)
    return text    

In [9]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv(data_path)
df.columns = ['contents']
df = df[['contents']]
df["contents"] = df["contents"].str.replace(pat=r'[^a-zA-Z0-9ㄱ-ㅣ가-힣. ]', repl=r'', regex=True)
df["contents"] = df["contents"].str.replace(pat=r'[\n]', repl=r'', regex=True)
df_new = []
tmp = df['contents']
for i in range(len(tmp)):
    tmp2 = str(tmp[i]).split('.  ')
    tmp2 = list(filter(None, tmp2))
    tmp2 = [item + '.' for item in tmp2]
    df_new.extend(tmp2)
df_new = pd.DataFrame(df_new, columns=['contents'])
df_new

,contents
0,울고 있는 그 여잘 아나요그댈 기다리는 바보 같은 사람멀어져 간단 걸 알면서도그댈 ...
1,그 언젠가 별빛에 새긴 추억 하나가바람에 두고 온 너의 이름을 불러와너의 이름은 떠...
2,언제였죠 그대 내게 처음 오던 날아직도 난 설레는걸요눈이 부시던 그대 모습내 맘 흔...
3,어디까지 말을 할까어떡하면 맘을 알까기다릴 수도 더 버틸 수도 없는무심히 나를 보네...
4,언젠가는 그대를 잊을 때도 오겠지오랜 시간 지나면애써 그댈 지우려 하지는 않아도 되...
...,...
995,오늘일줄야 늘생각은 했지만이른거아냐 좀서운하다네가 했던 따뜻한 말의 온도여기 그대론...
996,집으로 가는 밤길꿈도 잠이 든 이 밤가로등 빛 아래공허하게 날아드는 하루살이몸을 누...
997,내일이 되면널 사랑하게 될 거야예정된 일인 걸불안한 화음들과부딪는 현의 떨림점점 선...
998,You called me your loverYou called me your fri...


In [10]:
import tqdm
for i in tqdm.tqdm(range(len(df_new))):
    df_new['contents'][i] = cleasing(df_new['contents'][i])
    

100%|██████████| 1000/1000 [00:00<00:00, 9787.20it/s]


In [11]:
df_new.to_csv(model_save)

# 모델링

In [12]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token = '</s>', eos_token = '</s>', unk_token = '<unk>',
                pad_token = '<pad>', mask_token = '<mask>')
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir, legacy_format = False)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir = output_dir,
        overwrite_output_dir = overwrite_output_dir,
        per_device_eval_batch_size = per_device_train_batch_size,
        num_train_epochs = num_train_epochs,
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset,
    )

    trainer.train()
    trainer.save_model()

In [13]:
model_name = 'skt/kogpt2-base-v2'
output_dir = output_path
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

In [ ]:
train_file_path = model_save
train(train_file_path,
    model_name = model_name,
    output_dir = output_dir,
    overwrite_output_dir = overwrite_output_dir,
    per_device_train_batch_size = per_device_train_batch_size,
    num_train_epochs = num_train_epochs,
    save_steps = save_steps
)

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(sequence, max_lenth):
  model_path =output_path
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence},', return_tensors = 'pt')
  final_outputs = model.generate(
      ids,
      do_sample = True,
      max_length = max_length,
      pad_token_id = model.config.pad_token_id,
      repetition_penalty = 1.5, # 단어 빈도 패널티
      no_repeat_ngram_size=2, # 반복 줄이기 n개 이상의 토큰이 반복될 경우 등장확률을 0으로 만듬
      temperature=0.9, # 모델의 다음 토큰 확률분포에 변형을 줘 문장을 다양하게 생성
      tok_k = 50, # 가장 높은 확률을 지닌 n개의 단어수 중 추출
      top_p = 0.9 # 누적확률이 n%인 단어까지 포함하여 추출
  )
  result = tokenizer.decode(final_outputs[0], skip_special_tokens = True)
  result = re.sub(pattern='\n[0-9][0-9][0-9][0-9],', repl='', string=result)
  result = re.sub(pattern='\n[0-9][0-9][0-9],', repl='', string=result)
  result = re.sub(pattern='  ', repl=' ', string=result)
  result = re.sub(pattern=',', repl=' ', string=result)
  result = re.sub(pattern='  ', repl=' ', string=result)
  return result

In [ ]:
input = '제주 애월읍 해변위에 앉아 사진을 찍었다.'
sequence = input
max_length = 128 
print('input :' + sequence)
result = generate_text(sequence, max_length)
result

loading configuration file result/브런치_임재건시_3000\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type"

input :제주 애월읍 해변위에 앉아 사진을 찍었다.


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at result/브런치_임재건시_3000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file result/브런치_임재건시_3000\added_tokens.json. We won't load it.
loading file None
loading file result/브런치_임재건시_3000\special_tokens_map.json
loading file result/브런치_임재건시_3000\tokenizer_config.json
loading file result/브런치_임재건시_3000\tokenizer.json


'제주 애월읍 해변위에 앉아 사진을 찍었다. 사진 찍는 것도 일이라지만 저기서는 제 집도 못 찾은 어린 숨의 집을 보여 주고 싶었다.당신은 나를 꽃처럼 피우시어 당신이 나보다 고운 까닭으로 비에 시리다 지다가 구름 사이 띄운 하얀 잎 붉게 물든 봄날 좋은 날 좋다 하시었네.5월의 여름은 푸르렀어도 한겨울 눈 위로 붉었을까요.아무렴 어때. 푸른 하늘 나도 붉은 빛이야.7월에 오신다는 손님 어른 아이 이름 다 불러줘서 고맙'

In [ ]:
output_result = []
for _ in range(5):
    tmp1 = result.strip(input)
    tmp1 = tmp1.strip(', ')
    tmp1 = tmp1.strip(',. ')
    tmp2 = tmp1.split('.')
    input = tmp2[0]+'.'
    input = input.replace('\n', '')
    max_length = 128 
    output_result.append(input)
    globals()['result'] = generate_text(input, max_length)
output_result

loading configuration file result/브런치_임재건시_3000\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type"

['는 것도 일이라지만 저기서는 제 집도 못 찾은 어린 숨의 집을 보여 주고 싶었다.',
 '세상에 없는 그런 집은 여럿일 수 없어.',
 '다만 나는 그렇게 있잖아.',
 '러니까 너도 네 집인 줄 아느냐 묻지 마.',
 '내 사랑 잘 알아서 모두 내 탓이야.']

In [ ]:
input = '제주 애월읍 해변위에 앉아 사진을 찍었다. 행복함.'
sequence = input
max_length = 128 
print('input :' + sequence)
result = generate_text(sequence, max_length).strip(r'[^a-zA-Z0-9ㄱ-ㅣ가-힣. ]')
result = re.sub(pattern='\n[0-9][0-9][0-9][0-9],', repl='', string=result)
result

loading configuration file result/브런치_임재건시_3000\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type"

input :제주 애월읍 해변위에 앉아 사진을 찍었다. 행복함.


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at result/브런치_임재건시_3000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file result/브런치_임재건시_3000\added_tokens.json. We won't load it.
loading file None
loading file result/브런치_임재건시_3000\special_tokens_map.json
loading file result/브런치_임재건시_3000\tokenizer_config.json
loading file result/브런치_임재건시_3000\tokenizer.json


'제주 애월읍 해변위에 앉아 사진을 찍었다. 행복함. 우리가 행복한 줄도 알아.오래 참회가 희망인가. 다시 한 번 소회를 물으니 나는 그게 무죄다 하는 편이 낫다.5분만이라 하여도 못 들은 척 하고 일어나듯 세심하게 살핀 후 다음 기회에 넘긴다.내놓는 글이 너 하나뿐인 아침이 반갑다고 읽히면 좋겠어.나는 지난번 담장에 꽃밭이던 시절에 본 것을 시작으로 모두 오래된 것이라며 새로 본다.꽃처럼 피었다가 잎대로 벌건 낯 붉히네'

In [ ]:
output_result = []
for i in range(5):
    tmp1 = result.strip(input)
    tmp1 = tmp1.strip(',')
    tmp1 = tmp1.strip(', ')
    tmp1 = tmp1.strip(',. ')
    tmp2 = tmp1.split('.')
    input = tmp2[0]+'.'
    input = input.replace('\n', '')
    max_length = 128 
    output_result.append(input)
    globals()['result'] = generate_text(input, max_length)
output_result

loading configuration file result/브런치_임재건시_3000\config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type"

['우리가 행복한 줄도 알아.',
 '나는 달을 본 적 없는 사람인데 내가 아는 달은 다른 별에 비할까 작은 별이 많았으나 나는 불행하지 않았다.',
 '니 다행이었다 할 수 있는 것이 아니었어.',
 '살아보고 또 살아서 얼마든 행복할 것 같았어.',
 '5월의 봄은 어떤 꽃을 피우려고 초승달인가 보다.']